# Demo using Doppelgangers for SFM disambiguation

This notebook shows how to use Doppelgangers model for disambiguate and correct SFM reconstruction on highly symmetric scenes with COLMAP.

### Overview:
- COLMAP feature extraction and matching
- Extracting LoFTR matches
- Running Doppelgangers classifiers on image pairs
- Clean COLMAP database by removing pairs under the threshold
- COLMAP reconstruction with clean database

In [1]:
import numpy as np
import os
os.chdir("..")
import yaml

from doppelgangers.utils.process_database import create_image_pair_list, remove_doppelgangers
from doppelgangers.utils.loftr_matches import save_loftr_matches

In [2]:
# ------------ input setting -------------
input_image_path = 'data/sfm_disambiguation/yan2017/cup/images'
colmap_exe_command = 'colmap' # command to call Colmap in the command-line interface
matching_type = 'vocab_tree_matcher' # or 'exhaustive_matcher'
# ----------------------------------------
if matching_type == 'vocab_tree_matcher':
    # efficient for large-scale scenes
    vocab_tree_path = 'weights/vocab_tree_flickr100K_words1M.bin'
    if not os.path.exists(vocab_tree_path):
        os.system('wget https://demuc.de/colmap/vocab_tree_flickr100K_words1M.bin')
        os.system('mv vocab_tree_flickr100K_words1M.bin weights/')

# ------------ output setting -------------
output_path = 'results/cup/'
# -----------------------------------------
database_path = os.path.join(output_path, 'database.db')
colmap_result_path = os.path.join(output_path, 'sparse')
doppelgangers_result_path = os.path.join(output_path, 'sparse_doppelgangers')
loftr_matches_path = os.path.join(output_path, 'loftr_match')
config_file = os.path.join(output_path, 'config.yaml')
os.makedirs(output_path, exist_ok=True)
os.makedirs(colmap_result_path, exist_ok=True)
os.makedirs(doppelgangers_result_path, exist_ok=True)
os.makedirs(loftr_matches_path, exist_ok=True)

# ------------ threshold setting -------------
# doppelgangers threshold: smaller means more pairs will be included, larger means more pairs will be filtered out
# when the reconstruction is split into several components, threshold can be set to smaller
# when the reconstruction is not fully disambiguated, threshold can be set to larger
threshold = 0.8
# --------------------------------------------

### COLMAP feature extraction and matching

In [4]:
# feature extraction
command = [colmap_exe_command, 'feature_extractor', 
           '--image_path', input_image_path,
           '--database_path', database_path
          ]
os.system(' '.join(command))


Feature extraction

Processed file [1/65]
  Name:            0000.jpg
  Dimensions:      1067 x 800
  Camera:          #1 - SIMPLE_RADIAL
  Focal Length:    731.66px (Prior)
  Features:        995
Processed file [2/65]
  Name:            0001.jpg
  Dimensions:      1067 x 800
  Camera:          #1 - SIMPLE_RADIAL
  Focal Length:    731.66px (Prior)
  Features:        1153
Processed file [3/65]
  Name:            0002.jpg
  Dimensions:      1067 x 800
  Camera:          #1 - SIMPLE_RADIAL
  Focal Length:    731.66px (Prior)
  Features:        1042
Processed file [4/65]
  Name:            0003.jpg
  Dimensions:      1067 x 800
  Camera:          #1 - SIMPLE_RADIAL
  Focal Length:    731.66px (Prior)
  Features:        994
Processed file [5/65]
  Name:            0004.jpg
  Dimensions:      1067 x 800
  Camera:          #1 - SIMPLE_RADIAL
  Focal Length:    731.66px (Prior)
  Features:        902
Processed file [6/65]
  Name:            0005.jpg
  Dimensions:      1067 x 800
  Camera:   

0

In [8]:
# feature matching
command = [colmap_exe_command, matching_type,
           '--database_path', database_path
           ]    
if matching_type == 'vocab_tree_matcher':
    command += ['--VocabTreeMatching.vocab_tree_path', vocab_tree_path 
               ]    
    
os.system(' '.join(command))


Vocabulary tree feature matching

Indexing image [1/64] in 0.415s
Indexing image [2/64] in 0.332s
Indexing image [3/64] in 0.305s
Indexing image [4/64] in 0.307s
Indexing image [5/64] in 0.313s
Indexing image [6/64] in 0.284s
Indexing image [7/64] in 0.270s
Indexing image [8/64] in 0.246s
Indexing image [9/64] in 0.248s
Indexing image [10/64] in 0.238s
Indexing image [11/64] in 0.277s
Indexing image [12/64] in 0.264s
Indexing image [13/64] in 0.257s
Indexing image [14/64] in 0.253s
Indexing image [15/64] in 0.263s
Indexing image [16/64] in 0.280s
Indexing image [17/64] in 0.286s
Indexing image [18/64] in 0.305s
Indexing image [19/64] in 0.260s
Indexing image [20/64] in 0.229s
Indexing image [21/64] in 0.254s
Indexing image [22/64] in 0.263s
Indexing image [23/64] in 0.252s
Indexing image [24/64] in 0.248s
Indexing image [25/64] in 0.250s
Indexing image [26/64] in 0.265s
Indexing image [27/64] in 0.285s
Indexing image [28/64] in 0.313s
Indexing image [29/64] in 0.309s
Indexing image [3

0

### Extracting LoFTR matches

In [3]:
pair_path = create_image_pair_list(database_path, output_path)
save_loftr_matches(input_image_path, pair_path, output_path)

### Running Doppelgangers classifier model on image pairs

In [32]:
# edit config file with corresponding data path
example_config_file = 'doppelgangers/configs/test_configs/sfm_disambiguation_example.yaml'
with open(example_config_file) as f:
    example_config = yaml.safe_load(f)
    
example_config['data']['image_dir'] = input_image_path
example_config['data']['loftr_match_dir'] = loftr_matches_path
example_config['data']['test']['pair_path'] = pair_path
example_config['data']['output_path'] = output_path

with open(config_file, "w") as f:
    yaml.dump(example_config, f)

In [ ]:
# running doppelgangers classifier model on image pairs
command = ['python test_sfm_disambiguation.py', 
           config_file,
           '--pretrained', 'weights/doppelgangers_classifier_loftr.pt'
          ]
os.system(' '.join(command))

### Remove doppelganger pairs from COLMAP database with a threshold

In [4]:
# remove all the pairs with a probability lower than the threshold
from doppelgangers.utils.process_database import remove_doppelgangers
pair_probability_file = os.path.join(output_path, "pair_probability_list.npy")
update_database_path = remove_doppelgangers(database_path, pair_probability_file, pair_path, threshold)

2016 2016
Total 1891 Records deleted successfully
Total 1891 Records deleted successfully


### COLMAP sparse reconstruction

In [5]:
# colmap reconstruction with doppelgangers 
command = [colmap_exe_command, 'mapper',
           '--database_path', update_database_path,
           '--image_path', input_image_path,
           '--output_path', doppelgangers_result_path
          ]
os.system(' '.join(command)) 


Loading database

Loading cameras... 1 in 0.002s
Loading matches... 125 in 0.029s
Loading images... 64 in 0.112s (connected 63)
Building correspondence graph... in 0.007s (ignored 0)

Elapsed time: 0.003 [minutes]


Finding good initial image pair


Initializing with image pair #11 and #8


Global bundle adjustment

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  5.426145e+01    0.00e+00    5.52e+02   0.00e+00   0.00e+00  1.00e+04        0    1.67e-03    4.71e-02
   1  3.017475e+01    2.41e+01    2.55e+02   4.94e-01   9.99e-01  3.00e+04        1    2.06e-03    4.93e-02
   2  3.004198e+01    1.33e-01    1.15e+02   1.26e+00   1.03e+00  9.00e+04        1    7.45e-04    5.00e-02
   3  3.003547e+01    6.50e-03    3.02e+02   4.06e+00   1.69e-01  6.97e+04        1    7.37e-04    5.08e-02
   4  2.998961e+01    4.59e-02    2.04e+02   3.27e+00   8.00e-01  8.89e+04        1    7.32e-04    5.16e-02
   5  2.997686e+01    1.27e-02   

0

In [6]:
# colmap reconstruction  
command = [colmap_exe_command, 'mapper',
           '--database_path', database_path,
           '--image_path', input_image_path,
           '--output_path', colmap_result_path
          ]
os.system(' '.join(command)) 


Loading database

Loading cameras... 1 in 0.002s
Loading matches... 2016 in 0.211s
Loading images... 64 in 0.116s (connected 64)
Building correspondence graph... in 0.048s (ignored 0)

Elapsed time: 0.006 [minutes]


Finding good initial image pair


Initializing with image pair #42 and #46


Global bundle adjustment

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  1.747410e+01    0.00e+00    8.66e+02   0.00e+00   0.00e+00  1.00e+04        0    4.14e-04    1.25e-03
   1  1.503742e+01    2.44e+00    3.86e+02   1.23e+00   9.52e-01  3.00e+04        1    7.48e-04    2.04e-03
   2  1.447184e+01    5.66e-01    3.87e+01   7.06e-01   1.02e+00  9.00e+04        1    6.36e-04    2.71e-03
   3  1.442182e+01    5.00e-02    1.53e+02   8.04e-01   1.06e+00  2.70e+05        1    6.30e-04    3.36e-03
   4  1.444761e+01   -2.58e-02    1.53e+02   4.87e+00  -1.92e+00  1.35e+05        1    2.63e-04    3.64e-03
   5  1.441534e+01    6.48e-03 

0

In [ ]:
print("COLMAP results: %s \n disambiguated results: %s" % (colmap_result_path, doppelgangers_result_path))